# 参考

In [ ]:
# https://zenn.dev/mst8823/articles/da505dcf45474f

# Pipeline

## Config

In [ ]:
class Config:
    from requests import get
    name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].split('.')[0]  # notebookのタイトル取得
    only_inference = False
    if only_inference:
        task = 'infer'
    else:
        task = 'train'

    # クロスバリデーション設定
    n_fold = 2
    trn_fold = range(n_fold)

    seed = 42
    
    target_col = "point_of_interest"  # ⚠️コンペごとに更新
    # debug = False


    # lgb params
    def get_lgb_params() -> dict:
        lgb_params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': 64,
            'min_data_in_leaf': 20,
            'max_depth': 7,
            'verbose': 0
            }
        return lgb_params
    """
    # DNN params
    model_name = "roberta-base"
    learning_rate = 1e-5
    max_length = 256
    epochs = 8
    batch_size = 16
    """
    # knn params
    n_neighbors = 10

    # 解凍ファイル
    zip_file = 'foursquare-location-matching.zip'  # ⚠️コンペごとに更新

    # Colab Env
    upload_from_colab = True
    api_path = "/content/drive/MyDrive/kaggle/kaggle.json"
    drive_path = "/content/drive/MyDrive/kaggle/Foursquare"  # ⚠️コンペごとに更新
    
    # Kaggle Env
    kaggle_input_path = "../input/jpx-tokyo-stock-exchange-prediction"  # ⚠️コンペごとに更新
    kaggle_dataset_path = None

## Library

In [ ]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import pytz
import sys
import re

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, mean_squared_error

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras import backend as K
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.utils import plot_model

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import pickle
import glob

# import shap
import xgboost
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

warnings.filterwarnings("ignore")

## Utils

In [ ]:
# log を txtファイルに出力させるためのクラス
class Logger:
    # 参考) https://github.com/ghmagazine/kagglebook/blob/master/ch04-model-interface/code/util.py
    def __init__(self, path, log_title='Experiment'):
        self.general_logger = logging.getLogger(path)
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler(os.path.join(path, f'{log_title}.log'))
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)

    def info(self, message):
        # display time
        self.general_logger.info(f'[{self.now_string()}] - {message}')

    @staticmethod
    def now_string():
        return str(datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# シード固定用関数
def seed_everything(seed=42):
#  参考) https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

## SetUp

### 環境ごとのセットアップ

In [ ]:
COLAB = "google.colab" in sys.modules

In [ ]:
# -------------------------------colab 環境の場合-------------------------------
if COLAB:
    print("-------------------------------This environment is Google Colab-------------------------------")
    
    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive') 

    # my-modules のPath設定
    import sys
    sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')

    # use kaggle api (need kaggle token)
    f = open(Config.api_path, 'r')
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    
    # set dirs
    DRIVE = Config.drive_path
    EXP = (Config.name if Config.name is not None 
           else get("http://172.28.0.2:9000/api/sessions").json()[0]["name"][:-6])
    INPUT = os.path.join(DRIVE, "INPUT")
    FEATURES = os.path.join(INPUT, "FEATURES")
    OUTPUT = os.path.join(DRIVE, "OUTPUT")
    SUBMISSION = os.path.join(DRIVE, "SUBMISSION")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP) 
    EXP_MODEL = os.path.join(OUTPUT_EXP, "MODEL")
    EXP_FIG = os.path.join(OUTPUT_EXP, "FIG")
    EXP_PREDS = os.path.join(OUTPUT_EXP, "PREDS")

    # make dirs
    for d in [INPUT, FEATURES, SUBMISSION, EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)

    if not os.path.isfile(os.path.join(INPUT, Config.zip_file)):
        # download dataset
        # kaggle をインストール
        # アクセスパーミッションのため、以下を打ち込みます。
        ! chmod 600 /root/.kaggle/kaggle.json
        ! pip install kaggle
        ! kaggle competitions download -c foursquare-location-matching -p $INPUT  # ⚠️コンペごとに更新
        # 上記でdownloadしてきたZIPファイルを解凍
        ! apt-get install p7zip-full -y
        ! 7za x os.path.join(INPUT, Config.zip_file)
    else:
        print('DS for competition has been already installed.') 
    
    # utils
    logger = Logger(OUTPUT_EXP)
    
    sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')


# -------------------------------kaggle 環境の場合-------------------------------
else:
    print("-------------------------------This environment is Kaggle Kernel-------------------------------")
    
    # set dirs
    INPUT = Config.kaggle_input_path  # ⚠️コンペごとに更新
    EXP, OUTPUT, SUBMISSION = "./", "./", "./"
    EXP_MODEL = os.path.join(EXP, "model")
    EXP_FIG = os.path.join(EXP, "fig")
    EXP_PREDS = os.path.join(EXP, "preds")
    
    # copy dirs
    if Config.kaggle_dataset_path is not None:
        KD_MODEL = os.path.join(Config.kaggle_dataset_path, "model")
        KD_EXP_PREDS = os.path.join(Config.kaggle_dataset_path, "preds")
        shutil.copytree(KD_MODEL, EXP_MODEL)
        shutil.copytree(KD_EXP_PREDS, EXP_PREDS)

    # make dirs
    for d in [EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)
        
    # utils
    logger = Logger(EXP)

# utils
warnings.filterwarnings("ignore")
sns.set(style='whitegrid')
seed_everything(seed=Config.seed)

-------------------------------This environment is Google Colab-------------------------------
DS for competition has been already installed.


### input dataの読み込み

In [ ]:
# load data
raw_train = pd.read_csv('/content/drive/MyDrive/kaggle/Foursquare/INPUT/train.csv')
raw_test = pd.read_csv('/content/drive/MyDrive/kaggle/Foursquare/INPUT/test.csv')
raw_pairs = pd.read_csv('/content/drive/MyDrive/kaggle/Foursquare/INPUT/pairs.csv')
raw_sample_submission = pd.read_csv('/content/drive/MyDrive/kaggle/Foursquare/INPUT/sample_submission.csv')

## Feature Engineering

### 特徴量管理

#### 管理イメージ

イメージ.jpg

#### 基底Classや関数等の設定

In [ ]:
# 必要なライブラリのimport
import re
import time
from abc import ABCMeta, abstractmethod
from pathlib import Path
from contextlib import contextmanager

import pandas as pd

In [ ]:
# 特徴量pkl保存にかかった時間を表示
@contextmanager
def timer(name):
    t0 = time.time()
    print(f'[{name}] start')
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [ ]:
# 基底クラスの設定
class Feature(metaclass=ABCMeta):
    prefix = ''
    suffix = ''
    dir = "."
    
    def __init__(self):
        self.name = self.__class__.__name__
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.train_path = Path(self.dir) / f'{self.name}_train.pkl'
        self.test_path = Path(self.dir) / f'{self.name}_test.pkl'
    
    def run(self):
        with timer(self.name):
            self.create_features()
            prefix = self.prefix + '_' if self.prefix else ''
            suffix = '_' + self.suffix if self.suffix else ''
            self.train.columns = prefix + self.train.columns + suffix
            self.test.columns = prefix + self.test.columns + suffix
        return self
    
    @abstractmethod
    def create_features(self):
        raise NotImplementedError
    
    def save(self):
        self.train.to_pickle(str(self.train_path))
        self.test.to_pickle(str(self.test_path))

In [ ]:
# 特徴量に関するメモ生成関数
import csv
def create_memo(col_name, desc):
    file_path = os.path.join(FEATURES, "【Memo】Features.csv")

    if not os.path.isfile(file_path):
        with open(file_path, "w"): pass
        print('Memo was created.')

    with open(file_path, "r+") as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]

        col = [line for line in lines if line.split(",")[0] == col_name]
        if len(col) != 0: return

        writer = csv.writer(f)
        writer.writerow([col_name, desc])

In [ ]:
# 特徴量の既存判定・実行関数
import inspect

def get_features(namespace):
    for k, v in namespace.items():
        if inspect.isclass(v) and issubclass(v, Feature) and not inspect.isabstract(v):
            yield v()


def generate_features(namespace, overwrite=False):
    for f in get_features(namespace):
        if f.train_path.exists() and f.test_path.exists() and not overwrite:  # train/testのpklファイルが既存かつoverwrite=Falseなら保存しない。
            print(f.name, 'was skipped.')
        else:
            f.run().save()

In [ ]:
def load_datasets(feats):
    dfs = [pd.read_pickle(os.path.join(FEATURES, f'{f}_train.pkl')) for f in feats]
    X_train = pd.concat(dfs, axis=1).reset_index(drop=True)
    dfs = [pd.read_pickle(os.path.join(FEATURES, f'{f}_test.pkl')) for f in feats]
    X_test = pd.concat(dfs, axis=1).reset_index(drop=True)
    return X_train, X_test

In [ ]:
def create_class_names(df_name: str, class_name: str='Template')-> str: 
    return f"""
    class {class_name}(Feature):
        def create_features(self):
            self.train['{class_name}'] = {df_name}['{class_name}']
            self.test['{class_name}'] = {df_name}['{class_name}']
            create_memo('{class_name}', '~~~')
    """

def create_class_codes(df):
    for feat in list(df.columns):
        print(create_class_names('df', feat))

In [ ]:
# for feat in list(add_dist.columns):
#     print(create_class_names('add_dist', feat))

NameError: ignored

#### 各特徴量の定義

In [ ]:
# 特徴量の保存先指定
Feature.dir = FEATURES

In [ ]:
# 参照データの読み込み
%%time
add_dist = pd.read_pickle(os.path.join(FEATURES, 'add_distance_features.pkl'))

#### 特徴量クラスの定義

In [ ]:
class Template(Feature):
    def create_features(self):
        self.train['temp'] = add_dist['latitude'] + 1
        self.test['temp'] = add_dist['latitude'] + 1
        create_memo('Template', '特徴量クラス設計のテンプレート')

#### 特徴量ファイル・メモの生成・読み込みなど

In [ ]:
# 各特徴量pklファイルとメモの生成
generate_features(globals(), overwrite=False)

In [ ]:
# 各特徴量の読み込み
feats = ['Template',
         'latitude']

X_train, X_test = load_datasets(features)
X_train

# Train Models

### LGBM

In [ ]:
def make_lgb_ds(X, y):
    return lgb.Dataset(data=X, label=y, feature_name='auto')

In [ ]:
def train_lgbm(df, folds, params):
    models = list()
    
    for fold in range(folds):
        model_path = os.path.join(EXP_MODEL, f"{Config.name}-seed{Config.seed}-fold{fold}")
        # modelが保存されていない場合はtrainning
        if not os.path.isfile(model_path):
            # train, valid毎の入出力を用意
            X_train = df[df.fold != fold][["SecuritiesCode", "Open", "High", "Low", "Close"]]
            y_train = df[df.fold != fold][["Target"]]
            X_valid = df[df.fold == fold][["SecuritiesCode", "Open", "High", "Low", "Close"]]
            y_valid = df[df.fold == fold][["Target"]]
            # train, valid毎にdsへ格納
            train_ds = make_lgb_ds(X_train, y_train)
            valid_ds = make_lgb_ds(X_valid, y_valid)
            # modelの用意
            model = lgb.train(params=params, 
                              train_set=train_ds, 
                              num_boost_round=10000, 
                              early_stopping_rounds=20,
                              valid_sets=[valid_ds])
            # validモードの予測とRMSEを計算
            oof_preds = model.predict(X_valid)
            oof_score = np.sqrt(mean_squared_error(y_valid, oof_preds))
            # modelsへmodelを追加
            models.append(model)
            # fold毎のmodelをpklファイルとして保存
            pickle.dump(model, open(model_path, 'wb'))
            print(f"{Config.name}-seed{Config.seed}-fold{fold} has been saved.")
            # fold毎にモデル名とスコア(RMSE)を表示
            logger.info(f"model_name:{Config.name}-seed:{Config.seed}-fold:{fold}\
                        \n-X_cols:{X_train.columns.values}-y:{y_train.columns.values} >>>>> RMSE={oof_score}")
            print(f'fold_{fold} has finished.')
            print('-----------------------------')
        # 既に保存済みの場合は保存しない
        else:
            print(f'fold_{fold}: No model trained.')
        
    return models

### Run - prints rmse for each fold

In [ ]:
%%time
lgb_params = Config.get_lgb_params()
lgbm_models = train_lgbm(prices, folds=Config.n_fold, params=lgb_params)

[1]	valid_0's rmse: 0.0233821
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.0233819
[3]	valid_0's rmse: 0.0233816
[4]	valid_0's rmse: 0.0233812
[5]	valid_0's rmse: 0.0233812
[6]	valid_0's rmse: 0.0233811
[7]	valid_0's rmse: 0.0233812
[8]	valid_0's rmse: 0.0233811
[9]	valid_0's rmse: 0.0233811
[10]	valid_0's rmse: 0.023381
[11]	valid_0's rmse: 0.023381
[12]	valid_0's rmse: 0.023381
[13]	valid_0's rmse: 0.0233811
[14]	valid_0's rmse: 0.0233811
[15]	valid_0's rmse: 0.0233815
[16]	valid_0's rmse: 0.0233813
[17]	valid_0's rmse: 0.0233814
[18]	valid_0's rmse: 0.0233814
[19]	valid_0's rmse: 0.0233814
[20]	valid_0's rmse: 0.0233813
[21]	valid_0's rmse: 0.0233813
[22]	valid_0's rmse: 0.0233814
[23]	valid_0's rmse: 0.0233815
[24]	valid_0's rmse: 0.0233816
[25]	valid_0's rmse: 0.0233815
[26]	valid_0's rmse: 0.0233815
[27]	valid_0's rmse: 0.0233817
[28]	valid_0's rmse: 0.0233817
[29]	valid_0's rmse: 0.0233817
[30]	valid_0's rmse: 0.0233819
[31]	valid_0's rmse:

[2022-04-22 00:05:24] - model_name:baseline-lgbm001-seed:42-fold:0                        
-X_cols:['SecuritiesCode' 'Open' 'High' 'Low' 'Close']-y:['Target'] >>>>> RMSE=0.023380966734661824


[32]	valid_0's rmse: 0.0233821
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.023381
baseline-lgbm001-seed42-fold0 has been saved.
fold_0 has finished.
-----------------------------
[1]	valid_0's rmse: 0.0233897
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.0233891
[3]	valid_0's rmse: 0.0233885
[4]	valid_0's rmse: 0.0233883
[5]	valid_0's rmse: 0.0233883
[6]	valid_0's rmse: 0.0233882
[7]	valid_0's rmse: 0.0233881
[8]	valid_0's rmse: 0.0233875
[9]	valid_0's rmse: 0.0233873
[10]	valid_0's rmse: 0.0233873
[11]	valid_0's rmse: 0.0233872
[12]	valid_0's rmse: 0.0233871
[13]	valid_0's rmse: 0.0233872
[14]	valid_0's rmse: 0.0233873
[15]	valid_0's rmse: 0.0233873
[16]	valid_0's rmse: 0.0233871
[17]	valid_0's rmse: 0.0233871
[18]	valid_0's rmse: 0.0233868
[19]	valid_0's rmse: 0.0233869
[20]	valid_0's rmse: 0.0233867
[21]	valid_0's rmse: 0.0233867
[22]	valid_0's rmse: 0.0233868
[23]	valid_0's rmse: 0.0233866
[24]	valid_0's rmse: 0.0233865
[25]	valid

[2022-04-22 00:05:29] - model_name:baseline-lgbm001-seed:42-fold:1                        
-X_cols:['SecuritiesCode' 'Open' 'High' 'Low' 'Close']-y:['Target'] >>>>> RMSE=0.02338601232761834


baseline-lgbm001-seed42-fold1 has been saved.
fold_1 has finished.
-----------------------------
[1]	valid_0's rmse: 0.02336
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.0233595
[3]	valid_0's rmse: 0.023359
[4]	valid_0's rmse: 0.0233587
[5]	valid_0's rmse: 0.0233586
[6]	valid_0's rmse: 0.0233583
[7]	valid_0's rmse: 0.023358
[8]	valid_0's rmse: 0.0233579
[9]	valid_0's rmse: 0.0233577
[10]	valid_0's rmse: 0.0233574
[11]	valid_0's rmse: 0.0233574
[12]	valid_0's rmse: 0.0233573
[13]	valid_0's rmse: 0.0233574
[14]	valid_0's rmse: 0.0233572
[15]	valid_0's rmse: 0.0233571
[16]	valid_0's rmse: 0.023357
[17]	valid_0's rmse: 0.0233572
[18]	valid_0's rmse: 0.0233571
[19]	valid_0's rmse: 0.0233572
[20]	valid_0's rmse: 0.0233572
[21]	valid_0's rmse: 0.0233572
[22]	valid_0's rmse: 0.0233573
[23]	valid_0's rmse: 0.0233573
[24]	valid_0's rmse: 0.0233573
[25]	valid_0's rmse: 0.0233573
[26]	valid_0's rmse: 0.023357
[27]	valid_0's rmse: 0.023357
[28]	valid_0's rmse:

...

baseline-lgbm001-seed42-fold2 has been saved.
fold_2 has finished.
-----------------------------
[1]	valid_0's rmse: 0.0234114
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.0234109
[3]	valid_0's rmse: 0.0234105
[4]	valid_0's rmse: 0.02341
[5]	valid_0's rmse: 0.0234094
[6]	valid_0's rmse: 0.0234094
[7]	valid_0's rmse: 0.0234091
[8]	valid_0's rmse: 0.0234089
[9]	valid_0's rmse: 0.0234086
[10]	valid_0's rmse: 0.0234082
[11]	valid_0's rmse: 0.0234077
[12]	valid_0's rmse: 0.0234077
[13]	valid_0's rmse: 0.0234076
[14]	valid_0's rmse: 0.0234076
[15]	valid_0's rmse: 0.0234075
[16]	valid_0's rmse: 0.0234074
[17]	valid_0's rmse: 0.0234076
[18]	valid_0's rmse: 0.0234074
[19]	valid_0's rmse: 0.0234073
[20]	valid_0's rmse: 0.0234072
[21]	valid_0's rmse: 0.0234072
[22]	valid_0's rmse: 0.0234072
[23]	valid_0's rmse: 0.0234073
[24]	valid_0's rmse: 0.0234073
[25]	valid_0's rmse: 0.023407
[26]	valid_0's rmse: 0.0234068
[27]	valid_0's rmse: 0.0234067
[28]	valid_0's r

[2022-04-22 00:05:36] - model_name:baseline-lgbm001-seed:42-fold:3                        
-X_cols:['SecuritiesCode' 'Open' 'High' 'Low' 'Close']-y:['Target'] >>>>> RMSE=0.023406459218855234


baseline-lgbm001-seed42-fold3 has been saved.
fold_3 has finished.
-----------------------------
[1]	valid_0's rmse: 0.0234137
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.0234134
[3]	valid_0's rmse: 0.0234132
[4]	valid_0's rmse: 0.023413
[5]	valid_0's rmse: 0.0234132
[6]	valid_0's rmse: 0.0234132
[7]	valid_0's rmse: 0.0234133
[8]	valid_0's rmse: 0.0234132
[9]	valid_0's rmse: 0.0234133
[10]	valid_0's rmse: 0.0234133
[11]	valid_0's rmse: 0.0234131
[12]	valid_0's rmse: 0.0234133
[13]	valid_0's rmse: 0.023413
[14]	valid_0's rmse: 0.023413
[15]	valid_0's rmse: 0.0234129
[16]	valid_0's rmse: 0.0234128
[17]	valid_0's rmse: 0.0234129
[18]	valid_0's rmse: 0.0234127
[19]	valid_0's rmse: 0.0234127
[20]	valid_0's rmse: 0.0234126
[21]	valid_0's rmse: 0.0234125
[22]	valid_0's rmse: 0.0234126
[23]	valid_0's rmse: 0.0234127
[24]	valid_0's rmse: 0.0234128
[25]	valid_0's rmse: 0.0234129
[26]	valid_0's rmse: 0.0234129
[27]	valid_0's rmse: 0.0234128
[28]	valid_0's r

[2022-04-22 00:05:40] - model_name:baseline-lgbm001-seed:42-fold:4                        
-X_cols:['SecuritiesCode' 'Open' 'High' 'Low' 'Close']-y:['Target'] >>>>> RMSE=0.023412485983868365


baseline-lgbm001-seed42-fold4 has been saved.
fold_4 has finished.
-----------------------------
CPU times: user 1min 1s, sys: 0 ns, total: 1min 1s
Wall time: 18.1 s


# Make Predictions & Submit

In [ ]:
def pred_to_submission(pred):
    pred = pred.sort_values(by = "Prediction", ascending=False)
    pred.Rank = np.arange(0,2000)
    pred = pred.sort_values(by = "SecuritiesCode", ascending=True)
    pred.drop(["Prediction"],axis=1)
    submission = pred[["Date","SecuritiesCode","Rank"]]
    return submission

In [ ]:
if not COLAB:
    import jpx_tokyo_market_prediction as JTMP
    env = JTMP.make_env()
    iter_test = env.iter_test()

    for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
        prices["SecuritiesCode"] = enc.fit_transform(prices[["SecuritiesCode"]])

        X_test = prices[["SecuritiesCode", "Open", "High", "Low", "Close"]]
        lgbm_preds = list()
        for model in lgbm_models:
            lgbm_preds.append( model.predict(X_test) )
        lgbm_preds = np.mean(lgbm_preds, axis=0)

        sample_prediction["Prediction"] = lgbm_preds
        submission = pred_to_submission(sample_prediction)

        env.predict(submission)

In [ ]:
X_test = prices[["SecuritiesCode", "Open", "High", "Low", "Close"]]
y_pred = 
for model in lgbm_models:
    display(model.predict(X_test))

array([0.00058517, 0.00084053, 0.00065522, ..., 0.00051088, 0.00068093,
       0.00057949])

array([0.00057564, 0.00237632, 0.00047534, ..., 0.0006841 , 0.00016297,
       0.00039606])

array([0.00061323, 0.00079155, 0.00047528, ..., 0.00055913, 0.00044011,
       0.00052926])

array([0.00088106, 0.00091438, 0.00047131, ..., 0.0009105 , 0.00036797,
       0.00034625])

array([0.00056112, 0.00099036, 0.00039488, ..., 0.00060517, 0.00031836,
       0.0004349 ])

## Others

In [ ]:
# folderのディレクトリ構造可視化ツール
import pathlib
import glob
import os

def tree(path, layer=0, is_last=False, indent_current='　'):
    if not pathlib.Path(path).is_absolute():
        path = str(pathlib.Path(path).resolve())

    # カレントディレクトリの表示
    current = path.split('/')[::-1][0]
    if layer == 0:
        print('<'+current+'>')
    else:
        branch = '└' if is_last else '├'
        print('{indent}{branch}<{dirname}>'.format(indent=indent_current, branch=branch, dirname=current))

    # 下の階層のパスを取得
    paths = [p for p in glob.glob(path+'/*') if os.path.isdir(p) or os.path.isfile(p)]
    def is_last_path(i):
        return i == len(paths)-1

    # 再帰的に表示
    for i, p in enumerate(paths):

        indent_lower = indent_current
        if layer != 0:
            indent_lower += '　　' if is_last else '│　'

        if os.path.isfile(p):
            branch = '└' if is_last_path(i) else '├'
            print('{indent}{branch}{filename}'.format(indent=indent_lower, branch=branch, filename=p.split('/')[::-1][0]))
        if os.path.isdir(p):
            tree(p, layer=layer+1, is_last=is_last_path(i), indent_current=indent_lower)

In [ ]:
tree('/content/drive/MyDrive/kaggle/JPXTokyoStock')

<JPXTokyoStock>
　├<utilities>
　│　├submit
　│　└kaggle_utilities
　├<input>
　│　├stock_list.csv
　│　├<data_specifications>
　│　│　├trades_spec.csv
　│　│　├options_spec.csv
　│　│　├stock_list_spec.csv
　│　│　├stock_price_spec.csv
　│　│　└stock_fin_spec.csv
　│　├<example_test_files>
　│　│　├trades.csv
　│　│　├financials.csv
　│　│　├secondary_stock_prices.csv
　│　│　├stock_prices.csv
　│　│　├sample_submission.csv
　│　│　└options.csv
　│　├<jpx_tokyo_market_prediction>
　│　│　├competition.cpython-37m-x86_64-linux-gnu.so
　│　│　└__init__.py
　│　├<supplemental_files>
　│　│　├financials.csv
　│　│　├secondary_stock_prices.csv
　│　│　├stock_prices.csv
　│　│　├options.csv
　│　│　└trades.csv
　│　├<train_files>
　│　│　├financials.csv
　│　│　├options.csv
　│　│　├secondary_stock_prices.csv
　│　│　├stock_prices.csv
　│　│　└trades.csv
　│　└jpx-tokyo-stock-exchange-prediction.zip
　├<output>
　│　└<baseline-lgbm001>
　│　　　├<model>
　│　　　│　├baseline-lgbm001-seed42-fold0
　│　　　│　├baseline-lgbm001-seed42-fold1
　│　　　│　├baseline-lgbm001-seed42-fold2
　│　　　│　├baseline-lgb